In [47]:
import sys
!{sys.executable} -m pip install yfinance
!{sys.executable} -m pip install psycopg2-binary
!{sys.executable} -m pip install pykrx
!{sys.executable} -m pip install pymysql

In [48]:
import os
import pandas as pd
import numpy as np
import yfinance as yf
from sqlalchemy import create_engine, Column, Integer, String, Float, Date, ForeignKey, Boolean, Text, DateTime, UniqueConstraint, Index, BigInteger, DECIMAL
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
from sqlalchemy.sql import func
import ta
from ta.momentum import RSIIndicator
from ta.volatility import BollingerBands
from ta.trend import MACD, SMAIndicator
from datetime import datetime, timedelta
import time
import logging
import concurrent.futures
from tqdm import tqdm
from pykrx import stock

In [49]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

In [50]:
# 데이터베이스 연결 설정 (PostgreSQL 사용)
# SQLite를 사용하려면: 'sqlite:///finance_data.db'
DB_URI = 'mysql+pymysql://root@localhost:3306/finance_db'

engine = create_engine(DB_URI, echo=False, pool_size=20, max_overflow=0)
Base = declarative_base()
Session = sessionmaker(bind=engine)

In [51]:
# 연결테스트
try:
    with engine.connect() as conn:
        result = conn.execute("SELECT version();")
        for row in result:
            print("✅ 연결 성공! PostgreSQL 버전:", row[0])
except Exception as e:
    print("❌ 연결 실패:", e)

✅ 연결 성공! PostgreSQL 버전: 9.3.0


In [52]:
# 모델 정의
class Stock(Base):
    __tablename__ = 'stocks'
    
    stock_id = Column(Integer, primary_key=True)
    symbol = Column(String(20), unique=True, nullable=False, index=True)  # 종목 코드 (야후 파이낸스 형식)
    krx_code = Column(String(20), index=True)  # KRX 종목 코드
    name = Column(String(100), nullable=False, index=True)  # 종목명
    market = Column(String(20), index=True)  # 시장 구분 (KOSPI, KOSDAQ)
    sector = Column(String(100), index=True)  # 업종
    industry = Column(String(100))  # 세부 업종
    listing_date = Column(Date)  # 상장일
    delisting_date = Column(Date)  # 상장폐지일
    is_active = Column(Boolean, default=True)  # 활성 상태
    description = Column(Text)  # 회사 설명
    created_at = Column(DateTime, default=func.now())
    updated_at = Column(DateTime, default=func.now(), onupdate=func.now())

    # 관계 설정
    daily_prices = relationship("DailyPrice", back_populates="stock", cascade="all, delete-orphan")
    technical_indicators = relationship("TechnicalIndicator", back_populates="stock", cascade="all, delete-orphan")


In [53]:
class DailyPrice(Base):
    __tablename__ = 'daily_prices'
    
    price_id = Column(Integer, primary_key=True)
    stock_id = Column(Integer, ForeignKey('stocks.stock_id', ondelete='CASCADE'), nullable=False)
    date = Column(Date, nullable=False)
    open_price = Column(Float)
    high_price = Column(Float)
    low_price = Column(Float)
    close_price = Column(Float)
    adjusted_close = Column(Float)
    volume = Column(BigInteger)  # BigInteger로 변경: 거래량은 큰 숫자가 될 수 있음
    change = Column(Float)  # 전일 대비 변화액
    change_rate = Column(Float)  # 전일 대비 변화율 (%)
    created_at = Column(DateTime, default=func.now())
    updated_at = Column(DateTime, default=func.now(), onupdate=func.now())
    
    # 관계 설정
    stock = relationship("Stock", back_populates="daily_prices")
    
    # 인덱스 추가
    __table_args__ = (
        UniqueConstraint('stock_id', 'date', name='uix_stock_date'),
        Index('ix_daily_prices_date', 'date'),
    )

In [54]:
class TechnicalIndicator(Base):
    __tablename__ = 'technical_indicators'
    
    indicator_id = Column(Integer, primary_key=True)
    stock_id = Column(Integer, ForeignKey('stocks.stock_id', ondelete='CASCADE'), nullable=False)
    date = Column(Date, nullable=False)
    
    # 이동평균선
    ma5 = Column(Float)
    ma10 = Column(Float)
    ma20 = Column(Float)
    ma60 = Column(Float)
    ma120 = Column(Float)
    
    # 볼린저 밴드
    bb_upper = Column(Float)  # 상단 밴드
    bb_middle = Column(Float)  # 중간 밴드 (SMA)
    bb_lower = Column(Float)  # 하단 밴드
    bb_width = Column(Float)  # 밴드 폭
    
    # RSI
    rsi = Column(Float)
    
    # MACD
    macd = Column(Float)
    macd_signal = Column(Float)
    macd_hist = Column(Float)
    
    # 볼륨 관련
    volume_ma20 = Column(Float)  # 20일 평균 거래량
    volume_ratio = Column(Float)  # 현재 거래량 / 20일 평균 거래량
    
    # 캔들 패턴 (간단한 패턴만)
    is_doji = Column(Boolean)  # 도지 패턴
    is_hammer = Column(Boolean)  # 망치형 패턴
    
    # 시그널
    golden_cross = Column(Boolean)  # 골든 크로스 (5일선이 20일선 상향 돌파)
    death_cross = Column(Boolean)  # 데드 크로스 (5일선이 20일선 하향 돌파)
    bb_upper_touch = Column(Boolean)  # 볼린저 밴드 상단 터치
    bb_lower_touch = Column(Boolean)  # 볼린저 밴드 하단 터치
    
    created_at = Column(DateTime, default=func.now())
    updated_at = Column(DateTime, default=func.now(), onupdate=func.now())
    
    # 관계 설정
    stock = relationship("Stock", back_populates="technical_indicators")
    
    # 인덱스 추가
    __table_args__ = (
        UniqueConstraint('stock_id', 'date', name='uix_stock_indicator_date'),
        Index('ix_tech_indicators_date', 'date'),
        Index('ix_tech_indicators_rsi', 'rsi'),
        Index('ix_tech_indicators_volume_ratio', 'volume_ratio'),
    )

In [55]:
class MarketIndex(Base):
    __tablename__ = 'market_indices'
    
    index_id = Column(Integer, primary_key=True)
    market = Column(String(20), nullable=False, index=True)  # KOSPI, KOSDAQ 등
    date = Column(Date, nullable=False)
    open_index = Column(Float)
    high_index = Column(Float)
    low_index = Column(Float)
    close_index = Column(Float)
    volume = Column(Integer)
    change = Column(Float)
    change_rate = Column(Float)
    created_at = Column(DateTime, default=func.now())
    updated_at = Column(DateTime, default=func.now(), onupdate=func.now())
    
    # 인덱스 추가
    __table_args__ = (
        UniqueConstraint('market', 'date', name='uix_market_date'),
        Index('ix_market_indices_date', 'date'),
    )

In [56]:
class MarketStat(Base):
    __tablename__ = 'market_stats'
    
    stat_id = Column(Integer, primary_key=True)
    date = Column(Date, nullable=False)
    market = Column(String(20), nullable=False)  # KOSPI, KOSDAQ, ALL
    rising_stocks = Column(Integer)  # 상승 종목 수
    falling_stocks = Column(Integer)  # 하락 종목 수
    unchanged_stocks = Column(Integer)  # 보합 종목 수
    total_stocks = Column(Integer)  # 총 종목 수
    total_volume = Column(BigInteger)  # 총 거래량 (BigInteger로 변경)
    total_value = Column(DECIMAL(20,2))  # 총 거래대금 (NUMERIC으로 변경)
    created_at = Column(DateTime, default=func.now())
    updated_at = Column(DateTime, default=func.now(), onupdate=func.now())
    
    # 인덱스 추가
    __table_args__ = (
        UniqueConstraint('market', 'date', name='uix_market_stat_date'),
        Index('ix_market_stats_date', 'date'),
    )

In [57]:
# 데이터베이스 초기화 함수
def init_db():
    Base.metadata.create_all(engine)
    logger.info("데이터베이스 테이블이 생성되었습니다.")

In [58]:
# pykrx를 사용하여 한국 주식 종목 목록 가져오기
def get_korean_stock_symbols():
    try:
        today = datetime.now().strftime('%Y%m%d')
        
        # KOSPI 종목 가져오기
        kospi_tickers = stock.get_market_ticker_list(today, market="KOSPI")
        kospi_symbols = []
        
        for ticker in kospi_tickers:
            name = stock.get_market_ticker_name(ticker)
            kospi_symbols.append({
                'symbol': f'{ticker}.KS',  # 야후 파이낸스 형식으로 변환
                'krx_code': ticker,        # 원래 KRX 티커 저장
                'name': name,
                'market': 'KOSPI'
            })
        
        # KOSDAQ 종목 가져오기
        kosdaq_tickers = stock.get_market_ticker_list(today, market="KOSDAQ")
        kosdaq_symbols = []
        
        for ticker in kosdaq_tickers:
            name = stock.get_market_ticker_name(ticker)
            kosdaq_symbols.append({
                'symbol': f'{ticker}.KQ',  # 야후 파이낸스 형식으로 변환
                'krx_code': ticker,        # 원래 KRX 티커 저장
                'name': name,
                'market': 'KOSDAQ'
            })
        
        logger.info(f"종목 정보 가져오기 완료: KOSPI {len(kospi_symbols)}개, KOSDAQ {len(kosdaq_symbols)}개")
        return kospi_symbols + kosdaq_symbols
    except Exception as e:
        logger.error(f"종목 정보 가져오기 오류: {e}")
        # 오류 발생 시 샘플 데이터 반환
        kospi_symbols = [
            {'symbol': '005930.KS', 'krx_code': '005930', 'name': '삼성전자', 'market': 'KOSPI'},
            {'symbol': '000660.KS', 'krx_code': '000660', 'name': 'SK하이닉스', 'market': 'KOSPI'},
            {'symbol': '035420.KS', 'krx_code': '035420', 'name': 'NAVER', 'market': 'KOSPI'},
            {'symbol': '051910.KS', 'krx_code': '051910', 'name': 'LG화학', 'market': 'KOSPI'},
            {'symbol': '035720.KS', 'krx_code': '035720', 'name': '카카오', 'market': 'KOSPI'}
        ]
        
        kosdaq_symbols = [
            {'symbol': '293490.KQ', 'krx_code': '293490', 'name': '카카오게임즈', 'market': 'KOSDAQ'},
            {'symbol': '035900.KQ', 'krx_code': '035900', 'name': 'JYP Ent.', 'market': 'KOSDAQ'},
            {'symbol': '068270.KQ', 'krx_code': '068270', 'name': '셀트리온', 'market': 'KOSDAQ'},
            {'symbol': '259960.KQ', 'krx_code': '259960', 'name': '크래프톤', 'market': 'KOSDAQ'},
            {'symbol': '091990.KQ', 'krx_code': '091990', 'name': '셀트리온헬스케어', 'market': 'KOSDAQ'}
        ]
        
        logger.warning("pykrx 오류로 샘플 데이터를 사용합니다.")
        return kospi_symbols + kosdaq_symbols


In [59]:
# 주식 기본 정보 저장
def save_stock_info(symbols):
    session = Session()
    try:
        for symbol_info in tqdm(symbols, desc="종목 정보 저장"):
            symbol = symbol_info['symbol']
            krx_code = symbol_info['krx_code']
            name = symbol_info['name']
            market = symbol_info['market']
            
            # 이미 있는지 확인
            existing_stock = session.query(Stock).filter_by(symbol=symbol).first()
            
            if existing_stock:
                logger.debug(f"종목이 이미 존재합니다: {name} ({symbol})")
                continue
            
            # 야후 파이낸스에서 추가 정보 가져오기 시도
            try:
                stock_info = yf.Ticker(symbol).info
                sector = stock_info.get('sector', '')
                industry = stock_info.get('industry', '')
                description = stock_info.get('longBusinessSummary', '')
            except Exception as e:
                logger.warning(f"야후 파이낸스 정보 가져오기 실패: {symbol}, 오류: {e}")
                sector = ''
                industry = ''
                description = ''
            
            # 데이터베이스에 저장
            new_stock = Stock(
                symbol=symbol,
                krx_code=krx_code,
                name=name,
                market=market,
                sector=sector,
                industry=industry,
                description=description,
                is_active=True
            )
            
            session.add(new_stock)
            logger.debug(f"새 종목 추가: {name} ({symbol})")
        
        session.commit()
        logger.info(f"총 {len(symbols)}개 종목 정보가 저장/업데이트되었습니다.")
    except Exception as e:
        session.rollback()
        logger.error(f"종목 정보 저장 중 오류 발생: {e}")
    finally:
        session.close()


In [60]:
def fetch_stock_price(stock_id, symbol, start_date, end_date):
    try:
        # 야후 파이낸스에서 데이터 가져오기
        ticker = yf.Ticker(symbol)
        df = ticker.history(start=start_date, end=end_date, auto_adjust=False)
        
        if df is None or df.empty:
            logger.warning(f"가격 데이터가 없습니다: {symbol}")
            return []

        # 1행짜리 Series인 경우 DataFrame으로 변환
        if isinstance(df, pd.Series):
            df = df.to_frame().T

        # 데이터 처리
        df.reset_index(inplace=True)
        
        # 필요한 열이 모두 있는지 확인
        required_columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
        missing_columns = [col for col in required_columns if col not in df.columns]
        
        if missing_columns:
            logger.warning(f"필요한 열이 없습니다 ({symbol}): {missing_columns}")
            return []
        
        # 열 이름 변경
        rename_map = {
            'Date': 'date',
            'Open': 'open_price',
            'High': 'high_price',
            'Low': 'low_price',
            'Close': 'close_price',
            'Volume': 'volume'
        }
        
        # Adj Close가 있으면 추가
        if 'Adj Close' in df.columns:
            rename_map['Adj Close'] = 'adjusted_close'
        
        df.rename(columns=rename_map, inplace=True)
        
        # Adj Close가 없는 경우 Close로 대체
        if 'adjusted_close' not in df.columns:
            df['adjusted_close'] = df['close_price']
            logger.debug(f"Adj Close 없음, Close로 대체: {symbol}")
        
        # 변화율 계산 (수정종가 기준)
        df['change'] = df['adjusted_close'].diff()
        df['change_rate'] = df['adjusted_close'].pct_change() * 100
        
        # 결측치 처리
        df.fillna(0, inplace=True)
        
        # 결과 포맷팅
        result = []
        for _, row in df.iterrows():
            try:
                # 날짜 확인 및 변환
                if isinstance(row['date'], pd.Timestamp):
                    date_value = row['date'].date()
                elif isinstance(row['date'], datetime):
                    date_value = row['date'].date()
                else:
                    date_value = row['date']
                
                daily_price = {
                    'stock_id': stock_id,
                    'date': date_value,
                    'open_price': float(row['open_price']),
                    'high_price': float(row['high_price']),
                    'low_price': float(row['low_price']),
                    'close_price': float(row['close_price']),
                    'adjusted_close': float(row['adjusted_close']),
                    'volume': int(row['volume']),
                    'change': float(row['change']),
                    'change_rate': float(row['change_rate'])
                }
                result.append(daily_price)
            except Exception as e:
                logger.warning(f"행 처리 오류: {symbol}, {e}")
                continue
        
        if result:
            logger.info(f"가격 데이터 가져오기 성공: {symbol}, {len(result)}개 데이터")
        return result
    except Exception as e:
        logger.error(f"가격 데이터 가져오기 오류 ({symbol}): {e}")
        return []


In [61]:
# 주가 데이터 저장
def save_price_data(stock_id, price_data):
    session = Session()
    try:
        for data in price_data:
            # 이미 있는지 확인
            existing_price = session.query(DailyPrice).filter_by(
                stock_id=stock_id,
                date=data['date']
            ).first()
            
            if existing_price:
                # 기존 데이터 업데이트
                for key, value in data.items():
                    if key != 'stock_id' and key != 'date':
                        setattr(existing_price, key, value)
            else:
                # 새 데이터 추가
                new_price = DailyPrice(**data)
                session.add(new_price)
        
        session.commit()
        return True
    except Exception as e:
        session.rollback()
        logger.error(f"가격 데이터 저장 오류 (종목 ID: {stock_id}): {e}")
        return False
    finally:
        session.close()

In [62]:
from sqlalchemy.exc import OperationalError
import time, random

def calculate_and_save_with_retry(stock_id, max_retries=3):
    for attempt in range(1, max_retries+1):
        session = Session()
        try:
            # — 여기에 calculate_and_save_technical_indicators() 본문을 그대로 옮깁니다 —
            price_data = session.query(DailyPrice).filter_by(stock_id=stock_id).order_by(DailyPrice.date).all()
            if not price_data:
                logger.warning(f"기술적 지표 계산을 위한 가격 데이터가 없습니다 (종목 ID: {stock_id})")
                return False

            df = pd.DataFrame([{
                'date': p.date,
                'open': p.open_price,
                'high': p.high_price,
                'low': p.low_price,
                'close': p.adjusted_close,
                'volume': p.volume
            } for p in price_data])

            # 이동평균
            df['ma5']  = df['close'].rolling(5).mean()
            df['ma10'] = df['close'].rolling(10).mean()
            df['ma20'] = df['close'].rolling(20).mean()
            df['ma60'] = df['close'].rolling(60).mean()
            df['ma120']= df['close'].rolling(120).mean()

            # 볼린저 밴드
            bb = BollingerBands(close=df['close'], window=20, window_dev=2)
            df['bb_upper']  = bb.bollinger_hband()
            df['bb_middle'] = bb.bollinger_mavg()
            df['bb_lower']  = bb.bollinger_lband()
            df['bb_width']  = (df['bb_upper'] - df['bb_lower']) / df['bb_middle']

            # RSI
            df['rsi'] = RSIIndicator(close=df['close'], window=14).rsi()

            # MACD
            macd = MACD(close=df['close'], window_slow=26, window_fast=12, window_sign=9)
            df['macd']         = macd.macd()
            df['macd_signal']  = macd.macd_signal()
            df['macd_hist']    = macd.macd_diff()

            # 거래량 지표
            df['volume_ma20']  = df['volume'].rolling(20).mean()
            df['volume_ratio'] = df['volume'] / df['volume_ma20'] * 100

            # 캔들 패턴
            df['body_size']    = abs(df['close'] - df['open'])
            df['shadow_upper']= df['high'] - df[['open','close']].max(axis=1)
            df['shadow_lower']= df[['open','close']].min(axis=1) - df['low']
            df['is_doji']     = (df['body_size']/(df['high']-df['low']+1e-3) < 0.1)
            df['is_hammer']   = ((df['shadow_lower'] > 2*df['body_size']) &
                                 (df['shadow_upper'] < df['body_size']) &
                                 (df['body_size'] > 0))

            # 골든/데드 크로스
            df['ma5_prev']     = df['ma5'].shift(1)
            df['ma20_prev']    = df['ma20'].shift(1)
            df['golden_cross'] = (df['ma5'] > df['ma20']) & (df['ma5_prev'] <= df['ma20_prev'])
            df['death_cross']  = (df['ma5'] < df['ma20']) & (df['ma5_prev'] >= df['ma20_prev'])

            # 볼린저 터치
            df['bb_upper_touch']= df['high'] >= df['bb_upper']
            df['bb_lower_touch']= df['low']  <= df['bb_lower']

            df.fillna(0, inplace=True)

            # 기존 지표 삭제
            session.query(TechnicalIndicator).filter_by(stock_id=stock_id).delete()

            # 새 지표 삽입
            for _, row in df.iterrows():
                new_ind = TechnicalIndicator(
                    stock_id=stock_id, date=row['date'],
                    ma5=row['ma5'], ma10=row['ma10'], ma20=row['ma20'],
                    ma60=row['ma60'], ma120=row['ma120'],
                    bb_upper=row['bb_upper'], bb_middle=row['bb_middle'],
                    bb_lower=row['bb_lower'], bb_width=row['bb_width'],
                    rsi=row['rsi'], macd=row['macd'],
                    macd_signal=row['macd_signal'], macd_hist=row['macd_hist'],
                    volume_ma20=row['volume_ma20'], volume_ratio=row['volume_ratio'],
                    is_doji=row['is_doji'], is_hammer=row['is_hammer'],
                    golden_cross=row['golden_cross'], death_cross=row['death_cross'],
                    bb_upper_touch=row['bb_upper_touch'], bb_lower_touch=row['bb_lower_touch']
                )
                session.add(new_ind)

            session.commit()
            logger.debug(f"기술적 지표 계산 완료 (종목 ID: {stock_id})")
            return True

        except OperationalError as e:
            session.rollback()
            if 'Deadlock found' in str(e):
                wait = 0.5 + random.random() * 0.5
                logger.warning(f"Deadlock 발생, {wait:.2f}s 후 재시도합니다 ({attempt}/{max_retries})")
                time.sleep(wait)
                continue
            else:
                raise
        except Exception as e:
            session.rollback()
            logger.error(f"기술적 지표 계산 오류 (종목 ID: {stock_id}): {e}")
            return False
        finally:
            session.close()

    logger.error(f"{stock_id} 처리 재시도 실패")
    return False

In [63]:
def fetch_and_save_market_indices(start_date, end_date):
    indices = {
        'KOSPI': '^KS11',
        'KOSDAQ': '^KQ11'
    }
    
    session = Session()
    try:
        for market_name, symbol in indices.items():
            logger.info(f"{market_name} 지수 데이터 가져오는 중...")
            
            try:
                # 야후 파이낸스에서 데이터 가져오기
                df = yf.download(symbol, start=start_date, end=end_date, progress=False, auto_adjust=False)
                
                if df.empty:
                    logger.warning(f"{market_name} 지수 데이터가 없습니다.")
                    continue
                
                # 데이터 처리
                df.reset_index(inplace=True)
                df.rename(columns={
                    'Date': 'date',
                    'Open': 'open_index',
                    'High': 'high_index',
                    'Low': 'low_index',
                    'Close': 'close_index',
                    'Volume': 'volume'
                }, inplace=True)
                
                # 변화율 계산
                df['change'] = df['close_index'].diff()
                df['change_rate'] = df['close_index'].pct_change() * 100
                
                # 결측치 처리
                df.fillna(0, inplace=True)
                
                # 저장
                for _, row in df.iterrows():
                    # 날짜 변환 처리 - Series 객체 처리
                    date_value = None
                    if isinstance(row['date'], pd.Timestamp):
                        date_value = row['date'].date()
                    elif isinstance(row['date'], datetime):
                        date_value = row['date'].date()
                    elif isinstance(row['date'], pd.Series):
                        # Series인 경우 첫 번째 값을 가져와서 변환
                        date_series_value = row['date'].iloc[0] if len(row['date']) > 0 else None
                        if isinstance(date_series_value, pd.Timestamp):
                            date_value = date_series_value.date()
                        elif isinstance(date_series_value, datetime):
                            date_value = date_series_value.date()
                        else:
                            date_value = date_series_value
                    else:
                        date_value = row['date']
                    
                    # 날짜 값이 없으면 건너뛰기
                    if date_value is None:
                        logger.warning(f"날짜 값이 없습니다. 행을 건너뜁니다.")
                        continue
                    
                    # 데이터 값 추출 및 타입 변환
                    try:
                        open_index = float(row['open_index'])
                        high_index = float(row['high_index'])
                        low_index = float(row['low_index'])
                        close_index = float(row['close_index'])
                        volume = int(row['volume'])
                        change = float(row['change'])
                        change_rate = float(row['change_rate'])
                    except (ValueError, TypeError) as e:
                        logger.warning(f"데이터 변환 오류: {e}, 행을 건너뜁니다.")
                        continue
                    
                    # 이미 있는지 확인
                    existing_index = session.query(MarketIndex).filter_by(
                        market=market_name,
                        date=date_value
                    ).first()
                    
                    if existing_index:
                        # 기존 데이터 업데이트
                        existing_index.open_index = open_index
                        existing_index.high_index = high_index
                        existing_index.low_index = low_index
                        existing_index.close_index = close_index
                        existing_index.volume = volume
                        existing_index.change = change
                        existing_index.change_rate = change_rate
                    else:
                        # 새 데이터 추가
                        new_index = MarketIndex(
                            market=market_name,
                            date=date_value,
                            open_index=open_index,
                            high_index=high_index,
                            low_index=low_index,
                            close_index=close_index,
                            volume=volume,
                            change=change,
                            change_rate=change_rate
                        )
                        session.add(new_index)
                
                logger.info(f"{market_name} 지수 데이터 저장 완료")
            except Exception as e:
                logger.error(f"{market_name} 지수 데이터 처리 오류: {e}")
                continue
        
        session.commit()
        return True
    except Exception as e:
        session.rollback()
        logger.error(f"시장 지수 데이터 저장 오류: {e}")
        return False
    finally:
        session.close()

In [64]:
# 시장 통계 계산 및 저장 (수정됨)
def calculate_market_stats():
    session = Session()
    try:
        # 모든 날짜 가져오기
        dates = [date[0] for date in session.query(DailyPrice.date).distinct().all()]
        
        for date in tqdm(dates, desc="시장 통계 계산"):
            try:
                # KOSPI 통계
                kospi_stats = calculate_market_stat_for_date(session, date, 'KOSPI')
                # KOSDAQ 통계
                kosdaq_stats = calculate_market_stat_for_date(session, date, 'KOSDAQ')
                
                # 전체 시장 통계
                total_stats = {
                    'date': date,
                    'market': 'ALL',
                    'rising_stocks': kospi_stats['rising_stocks'] + kosdaq_stats['rising_stocks'],
                    'falling_stocks': kospi_stats['falling_stocks'] + kosdaq_stats['falling_stocks'],
                    'unchanged_stocks': kospi_stats['unchanged_stocks'] + kosdaq_stats['unchanged_stocks'],
                    'total_stocks': kospi_stats['total_stocks'] + kosdaq_stats['total_stocks'],
                    'total_volume': kospi_stats['total_volume'] + kosdaq_stats['total_volume'],
                    'total_value': float(kospi_stats['total_value']) + float(kosdaq_stats['total_value'])
                }
                
                # 저장
                save_market_stat(session, kospi_stats)
                save_market_stat(session, kosdaq_stats)
                save_market_stat(session, total_stats)
                
                # 정기적으로 커밋 (100개 날짜마다)
                if dates.index(date) % 100 == 0:
                    session.commit()
                    logger.debug(f"{date} 까지의 시장 통계 저장 완료")
            except Exception as e:
                logger.error(f"{date} 날짜 시장 통계 계산 오류: {e}")
                continue
        
        session.commit()
        logger.info("시장 통계 계산 및 저장 완료")
        return True
    except Exception as e:
        session.rollback()
        logger.error(f"시장 통계 계산 오류: {e}")
        return False
    finally:
        session.close()

In [65]:
# 특정 날짜, 특정 시장에 대한 통계 계산 (수정됨)
def calculate_market_stat_for_date(session, date, market):
    # 해당 시장의 모든 종목 가져오기
    stocks = session.query(Stock).filter_by(market=market).all()
    stock_ids = [stock.stock_id for stock in stocks]
    
    # 해당 날짜의 가격 데이터 가져오기
    prices = session.query(DailyPrice).filter(
        DailyPrice.stock_id.in_(stock_ids),
        DailyPrice.date == date
    ).all()
    
    # 통계 계산
    rising_stocks = sum(1 for p in prices if p.change > 0)
    falling_stocks = sum(1 for p in prices if p.change < 0)
    unchanged_stocks = sum(1 for p in prices if p.change == 0)
    total_stocks = len(prices)
    total_volume = sum(p.volume for p in prices)
    
    # 총 거래대금 계산 (부동소수점 오버플로우 방지)
    total_value = float(sum(float(p.volume) * float(p.close_price) for p in prices))
    
    return {
        'date': date,
        'market': market,
        'rising_stocks': rising_stocks,
        'falling_stocks': falling_stocks,
        'unchanged_stocks': unchanged_stocks,
        'total_stocks': total_stocks,
        'total_volume': total_volume,
        'total_value': total_value
    }

In [66]:
# 시장 통계 저장 (수정됨)
def save_market_stat(session, stat_data):
    try:
        # 이미 있는지 확인
        existing_stat = session.query(MarketStat).filter_by(
            market=stat_data['market'],
            date=stat_data['date']
        ).first()
        
        if existing_stat:
            # 기존 데이터 업데이트
            for key, value in stat_data.items():
                if key != 'market' and key != 'date':
                    setattr(existing_stat, key, value)
        else:
            # 새 데이터 추가
            new_stat = MarketStat(**stat_data)
            session.add(new_stat)
    except Exception as e:
        logger.error(f"시장 통계 저장 중 오류: {e}, 데이터: {stat_data}")
        raise

In [67]:
# 주식 데이터 가져오기 (병렬 처리)
def fetch_stock_data(start_date, end_date, max_workers=10):
    session = Session()
    try:
        # 모든 종목 가져오기
        stocks = session.query(Stock).all()
        
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = []
            
            for stock in stocks:
                futures.append(
                    executor.submit(
                        process_stock_data,
                        stock.stock_id,
                        stock.symbol,
                        stock.name,
                        start_date,
                        end_date
                    )
                )
            
            # 결과 처리
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(futures), desc="종목 데이터 처리"):
                try:
                    result = future.result()
                    logger.debug(f"종목 처리 완료: {result}")
                except Exception as e:
                    logger.error(f"종목 처리 오류: {e}")
        
        logger.info("모든 종목 데이터 처리 완료")
    finally:
        session.close()

In [68]:
# 단일 종목 데이터 처리
def process_stock_data(stock_id, symbol, name, start_date, end_date):
    logger.debug(f"종목 처리 시작: {name} ({symbol})")
    
    # 주가 데이터 가져오기
    price_data = fetch_stock_price(stock_id, symbol, start_date, end_date)
    
    if not price_data:
        logger.warning(f"가격 데이터 없음: {name} ({symbol})")
        return f"{name}: 데이터 없음"
    
    # 주가 데이터 저장
    save_result = save_price_data(stock_id, price_data)
    
    if not save_result:
        logger.error(f"가격 데이터 저장 실패: {name} ({symbol})")
        return f"{name}: 저장 실패"
    
    # 기술적 지표 계산 및 저장
    indicator_result = calculate_and_save_with_retry(stock_id, max_retries=3)
    
    if not indicator_result:
        logger.warning(f"기술적 지표 계산 실패: {name} ({symbol})")
    
    return f"{name}: 처리 완료 ({len(price_data)}개 데이터)"


In [69]:
# 데이터베이스 업데이트 (새로운 데이터 추가)
def update_database(days=30):
    today = datetime.now().date()
    start_date = (today - timedelta(days=days)).strftime('%Y-%m-%d')
    end_date = today.strftime('%Y-%m-%d')
    
    logger.info(f"데이터베이스 업데이트 시작 (기간: {start_date} ~ {end_date})")
    
    # 종목 정보 업데이트
    symbols = get_korean_stock_symbols()
    save_stock_info(symbols)
    
    # 주가 데이터 업데이트
    fetch_stock_data(start_date, end_date)
    
    # 시장 지수 업데이트
    fetch_and_save_market_indices(start_date, end_date)
    
    # 시장 통계 계산
    calculate_market_stats()
    
    logger.info("데이터베이스 업데이트 완료")


In [70]:
# 초기 데이터베이스 구축 (과거 데이터)
def build_initial_database(years=3):
    today = datetime.now().date()
    start_date = (today - timedelta(days=365 * years)).strftime('%Y-%m-%d')
    end_date = today.strftime('%Y-%m-%d')
    
    logger.info(f"초기 데이터베이스 구축 시작 (기간: {start_date} ~ {end_date})")
    
    # 데이터베이스 초기화
    init_db()
    
    # 종목 정보 저장
    symbols = get_korean_stock_symbols()
    save_stock_info(symbols)
    
    # 주가 데이터 가져오기 및 저장
    fetch_stock_data(start_date, end_date)
    
    # 시장 지수 가져오기 및 저장
    fetch_and_save_market_indices(start_date, end_date)
    
    # 시장 통계 계산
    calculate_market_stats()
    
    logger.info("초기 데이터베이스 구축 완료")


In [71]:
# RSI 과매수/과매도 종목 조회
def get_overbought_oversold_stocks(date, rsi_upper=70, rsi_lower=30):
    session = Session()
    try:
        # 과매수 종목 (RSI > rsi_upper)
        overbought = session.query(Stock, TechnicalIndicator).\
            join(TechnicalIndicator, Stock.stock_id == TechnicalIndicator.stock_id).\
            filter(TechnicalIndicator.date == date).\
            filter(TechnicalIndicator.rsi >= rsi_upper).\
            order_by(TechnicalIndicator.rsi.desc()).all()
        
        # 과매도 종목 (RSI < rsi_lower)
        oversold = session.query(Stock, TechnicalIndicator).\
            join(TechnicalIndicator, Stock.stock_id == TechnicalIndicator.stock_id).\
            filter(TechnicalIndicator.date == date).\
            filter(TechnicalIndicator.rsi <= rsi_lower).\
            order_by(TechnicalIndicator.rsi.asc()).all()
        
        return {
            'overbought': [{'name': s.name, 'symbol': s.symbol, 'rsi': t.rsi} for s, t in overbought],
            'oversold': [{'name': s.name, 'symbol': s.symbol, 'rsi': t.rsi} for s, t in oversold]
        }
    finally:
        session.close()


In [72]:
# 볼린저 밴드 상/하단 터치 종목 조회
def get_bollinger_band_touch_stocks(date):
    session = Session()
    try:
        # 상단 터치 종목
        upper_touch = session.query(Stock, TechnicalIndicator).\
            join(TechnicalIndicator, Stock.stock_id == TechnicalIndicator.stock_id).\
            filter(TechnicalIndicator.date == date).\
            filter(TechnicalIndicator.bb_upper_touch == True).all()
        
        # 하단 터치 종목
        lower_touch = session.query(Stock, TechnicalIndicator).\
            join(TechnicalIndicator, Stock.stock_id == TechnicalIndicator.stock_id).\
            filter(TechnicalIndicator.date == date).\
            filter(TechnicalIndicator.bb_lower_touch == True).all()
        
        return {
            'upper_touch': [{'name': s.name, 'symbol': s.symbol} for s, t in upper_touch],
            'lower_touch': [{'name': s.name, 'symbol': s.symbol} for s, t in lower_touch]
        }
    finally:
        session.close()


In [73]:
# 이동평균선 돌파 종목 조회
def get_ma_breakout_stocks(date, ma_type='ma20', percent=3):
    session = Session()
    try:
        # 해당 날짜의 데이터
        subq = session.query(Stock.stock_id, Stock.name, Stock.symbol, DailyPrice.close_price, getattr(TechnicalIndicator, ma_type)).\
            join(DailyPrice, Stock.stock_id == DailyPrice.stock_id).\
            join(TechnicalIndicator, (Stock.stock_id == TechnicalIndicator.stock_id) & (DailyPrice.date == TechnicalIndicator.date)).\
            filter(DailyPrice.date == date).subquery()
        
        # 이동평균선 대비 종가 비율 계산 및 필터링
        result = session.query(subq.c.name, subq.c.symbol, (subq.c.close_price / getattr(subq.c, ma_type) * 100 - 100).label('percent')).\
            filter(getattr(subq.c, ma_type) > 0).\
            filter((subq.c.close_price / getattr(subq.c, ma_type) * 100 - 100) >= percent).\
            order_by((subq.c.close_price / getattr(subq.c, ma_type) * 100 - 100).desc()).all()
        
        return [{'name': name, 'symbol': symbol, 'percent': round(pct, 2)} for name, symbol, pct in result]
    finally:
        session.close()

In [74]:
# 거래량 급증 종목 조회
def get_volume_surge_stocks(date, percent=100):
    session = Session()
    try:
        # 거래량 비율이 지정 퍼센트 이상인 종목
        result = session.query(Stock.name, Stock.symbol, TechnicalIndicator.volume_ratio).\
            join(TechnicalIndicator, Stock.stock_id == TechnicalIndicator.stock_id).\
            filter(TechnicalIndicator.date == date).\
            filter(TechnicalIndicator.volume_ratio >= percent).\
            order_by(TechnicalIndicator.volume_ratio.desc()).all()
        
        return [{'name': name, 'symbol': symbol, 'volume_ratio': round(ratio, 2)} for name, symbol, ratio in result]
    finally:
        session.close()


In [75]:
# 골든크로스/데드크로스 발생 종목 조회
def get_cross_stocks(start_date, end_date):
    session = Session()
    try:
        # 골든크로스
        golden_cross = session.query(Stock.name, Stock.symbol, TechnicalIndicator.date).\
            join(TechnicalIndicator, Stock.stock_id == TechnicalIndicator.stock_id).\
            filter(TechnicalIndicator.date.between(start_date, end_date)).\
            filter(TechnicalIndicator.golden_cross == True).all()
        
        # 데드크로스
        death_cross = session.query(Stock.name, Stock.symbol, TechnicalIndicator.date).\
            join(TechnicalIndicator, Stock.stock_id == TechnicalIndicator.stock_id).\
            filter(TechnicalIndicator.date.between(start_date, end_date)).\
            filter(TechnicalIndicator.death_cross == True).all()
        
        return {
            'golden_cross': [{'name': name, 'symbol': symbol, 'date': date} for name, symbol, date in golden_cross],
            'death_cross': [{'name': name, 'symbol': symbol, 'date': date} for name, symbol, date in death_cross]
        }
    finally:
        session.close()

In [76]:
# 특정 종목의 크로스 횟수 조회
def get_cross_count(symbol, start_date, end_date):
    session = Session()
    try:
        stock = session.query(Stock).filter_by(symbol=symbol).first()
        
        if not stock:
            return {'golden_cross': 0, 'death_cross': 0}
        
        # 골든크로스 횟수
        golden_count = session.query(func.count(TechnicalIndicator.indicator_id)).\
            filter(TechnicalIndicator.stock_id == stock.stock_id).\
            filter(TechnicalIndicator.date.between(start_date, end_date)).\
            filter(TechnicalIndicator.golden_cross == True).scalar()
        
        # 데드크로스 횟수
        death_count = session.query(func.count(TechnicalIndicator.indicator_id)).\
            filter(TechnicalIndicator.stock_id == stock.stock_id).\
            filter(TechnicalIndicator.date.between(start_date, end_date)).\
            filter(TechnicalIndicator.death_cross == True).scalar()
        
        return {'golden_cross': golden_count, 'death_cross': death_count}
    finally:
        session.close()

In [77]:
# 시장 통계 조회
def get_market_stats(date):
    session = Session()
    try:
        stats = session.query(MarketStat).filter_by(date=date).all()
        
        if not stats:
            return None
        
        result = {}
        for stat in stats:
            result[stat.market] = {
                'rising_stocks': stat.rising_stocks,
                'falling_stocks': stat.falling_stocks,
                'unchanged_stocks': stat.unchanged_stocks,
                'total_stocks': stat.total_stocks,
                'total_volume': stat.total_volume,
                'total_value': float(stat.total_value)  # NUMERIC을 float으로 변환
            }
        
        return result
    finally:
        session.close()

In [78]:
# 특정 종목의 가격 조회
def get_stock_price(symbol, date):
    session = Session()
    try:
        result = session.query(Stock.name, DailyPrice).\
            join(DailyPrice, Stock.stock_id == DailyPrice.stock_id).\
            filter(Stock.symbol == symbol).\
            filter(DailyPrice.date == date).first()
        
        if not result:
            return None
        
        name, price = result
        
        return {
            'name': name,
            'date': price.date.strftime('%Y-%m-%d'),
            'open': price.open_price,
            'high': price.high_price,
            'low': price.low_price,
            'close': price.close_price,
            'volume': price.volume,
            'change': price.change,
            'change_rate': price.change_rate
        }
    finally:
        session.close()

In [79]:
# 시장별 종목 수 조회
def get_market_stock_counts():
    session = Session()
    try:
        kospi_count = session.query(func.count(Stock.stock_id)).filter_by(market='KOSPI').scalar()
        kosdaq_count = session.query(func.count(Stock.stock_id)).filter_by(market='KOSDAQ').scalar()
        
        return {
            'KOSPI': kospi_count,
            'KOSDAQ': kosdaq_count,
            'TOTAL': kospi_count + kosdaq_count
        }
    finally:
        session.close()

In [80]:
# 시장 지수 조회
def get_market_index(market, date):
    session = Session()
    try:
        index = session.query(MarketIndex).\
            filter_by(market=market).\
            filter_by(date=date).first()
        
        if not index:
            return None
        
        return {
            'market': index.market,
            'date': index.date.strftime('%Y-%m-%d'),
            'open': index.open_index,
            'high': index.high_index,
            'low': index.low_index,
            'close': index.close_index,
            'volume': index.volume,
            'change': index.change,
            'change_rate': index.change_rate
        }
    finally:
        session.close()

In [81]:
# 시장별 거래량 상위 종목 조회
def get_top_volume_stocks(date, market, limit=10):
    session = Session()
    try:
        result = session.query(Stock.name, Stock.symbol, DailyPrice.volume).\
            join(DailyPrice, Stock.stock_id == DailyPrice.stock_id).\
            filter(Stock.market == market).\
            filter(DailyPrice.date == date).\
            order_by(DailyPrice.volume.desc()).\
            limit(limit).all()
        
        return [{'name': name, 'symbol': symbol, 'volume': volume} for name, symbol, volume in result]
    finally:
        session.close()

In [82]:
# 시장별 등락률 상위/하위 종목 조회
def get_top_change_rate_stocks(date, market, limit=10, top=True):
    session = Session()
    try:
        query = session.query(Stock.name, Stock.symbol, DailyPrice.change_rate).\
            join(DailyPrice, Stock.stock_id == DailyPrice.stock_id).\
            filter(Stock.market == market).\
            filter(DailyPrice.date == date)
        
        if top:
            # 상승률 상위
            result = query.order_by(DailyPrice.change_rate.desc()).limit(limit).all()
        else:
            # 하락률 상위
            result = query.order_by(DailyPrice.change_rate.asc()).limit(limit).all()
        
        return [{'name': name, 'symbol': symbol, 'change_rate': round(rate, 2)} for name, symbol, rate in result]
    finally:
        session.close()


In [83]:
# 시장별 가격 상위 종목 조회
def get_top_price_stocks(date, market, limit=10):
    session = Session()
    try:
        result = session.query(Stock.name, Stock.symbol, DailyPrice.close_price).\
            join(DailyPrice, Stock.stock_id == DailyPrice.stock_id).\
            filter(Stock.market == market).\
            filter(DailyPrice.date == date).\
            order_by(DailyPrice.close_price.desc()).\
            limit(limit).all()
        
        return [{'name': name, 'symbol': symbol, 'price': price} for name, symbol, price in result]
    finally:
        session.close()

In [84]:
# 이 함수를 사용하여 데이터베이스를 초기화하거나 업데이트합니다
def run_database_task(task="help", years=3, days=30):
    if task == "init":
        build_initial_database(years=years)
    elif task == "update":
        update_database(days=days)
    else:
        print("사용 가능한 작업:")
        print("- run_database_task('init', years=3) - 데이터베이스 초기화 및 구축")
        print("- run_database_task('update', days=30) - 데이터베이스 업데이트")

# 도움말 표시
run_database_task()

사용 가능한 작업:
- run_database_task('init', years=3) - 데이터베이스 초기화 및 구축
- run_database_task('update', days=30) - 데이터베이스 업데이트


In [85]:
# 초기화 및 구축
run_database_task("init", years=3)

2025-07-16 21:23:19,407 - __main__ - INFO - 초기 데이터베이스 구축 시작 (기간: 2022-07-17 ~ 2025-07-16)
2025-07-16 21:23:19,515 - __main__ - INFO - 데이터베이스 테이블이 생성되었습니다.
2025-07-16 21:23:20,196 - __main__ - INFO - 종목 정보 가져오기 완료: KOSPI 962개, KOSDAQ 1795개
종목 정보 저장: 100%|███████████████████████| 2757/2757 [24:31<00:00,  1.87it/s]
2025-07-16 21:47:52,163 - __main__ - INFO - 총 2757개 종목 정보가 저장/업데이트되었습니다.
종목 데이터 처리:   0%|                                | 0/2757 [00:00<?, ?it/s]2025-07-16 21:47:52,781 - __main__ - INFO - 가격 데이터 가져오기 성공: 00104K.KS, 732개 데이터
2025-07-16 21:47:52,809 - __main__ - INFO - 가격 데이터 가져오기 성공: 095570.KS, 732개 데이터
2025-07-16 21:47:52,826 - __main__ - INFO - 가격 데이터 가져오기 성공: 282330.KS, 732개 데이터
2025-07-16 21:47:52,930 - __main__ - INFO - 가격 데이터 가져오기 성공: 027410.KS, 732개 데이터
2025-07-16 21:47:52,949 - __main__ - INFO - 가격 데이터 가져오기 성공: 079160.KS, 732개 데이터
2025-07-16 21:47:53,001 - __main__ - INFO - 가격 데이터 가져오기 성공: 001040.KS, 732개 데이터
2025-07-16 21:47:53,006 - __main__ - INFO - 가격 데이터 가져오기 성공: 

종목 데이터 처리:   1%|▎                      | 30/2757 [00:19<32:16,  1.41it/s]2025-07-16 21:48:12,178 - __main__ - WARNING - Deadlock 발생, 0.89s 후 재시도합니다 (3/3)
2025-07-16 21:48:12,227 - __main__ - WARNING - Deadlock 발생, 0.84s 후 재시도합니다 (2/3)
2025-07-16 21:48:12,248 - __main__ - INFO - 가격 데이터 가져오기 성공: 007700.KS, 732개 데이터
종목 데이터 처리:   1%|▎                      | 31/2757 [00:20<30:38,  1.48it/s]2025-07-16 21:48:12,432 - __main__ - WARNING - Deadlock 발생, 0.88s 후 재시도합니다 (3/3)
2025-07-16 21:48:12,761 - __main__ - INFO - 가격 데이터 가져오기 성공: 114090.KS, 732개 데이터
종목 데이터 처리:   1%|▎                      | 32/2757 [00:20<27:39,  1.64it/s]2025-07-16 21:48:13,077 - __main__ - ERROR - 35 처리 재시도 실패
2025-07-16 21:48:13,079 - __main__ - WARNING - 기술적 지표 계산 실패: DS단석 (017860.KS)
종목 데이터 처리:   1%|▎                      | 34/2757 [00:20<17:05,  2.66it/s]2025-07-16 21:48:13,241 - __main__ - INFO - 가격 데이터 가져오기 성공: 078930.KS, 732개 데이터
2025-07-16 21:48:13,294 - __main__ - INFO - 가격 데이터 가져오기 성공: 006360.KS, 732개 데이터
2025-07-1

종목 데이터 처리:   3%|▌                      | 73/2757 [00:40<25:55,  1.73it/s]2025-07-16 21:48:32,763 - __main__ - INFO - 가격 데이터 가져오기 성공: 002380.KS, 732개 데이터
2025-07-16 21:48:32,962 - __main__ - WARNING - Deadlock 발생, 0.84s 후 재시도합니다 (2/3)
2025-07-16 21:48:33,105 - __main__ - WARNING - Deadlock 발생, 0.58s 후 재시도합니다 (1/3)
종목 데이터 처리:   3%|▌                      | 74/2757 [00:40<27:01,  1.65it/s]2025-07-16 21:48:33,106 - __main__ - WARNING - Deadlock 발생, 0.86s 후 재시도합니다 (2/3)
2025-07-16 21:48:33,372 - __main__ - WARNING - Deadlock 발생, 0.95s 후 재시도합니다 (1/3)
2025-07-16 21:48:33,415 - __main__ - INFO - 가격 데이터 가져오기 성공: 344820.KS, 732개 데이터
종목 데이터 처리:   3%|▋                      | 77/2757 [00:41<17:41,  2.52it/s]2025-07-16 21:48:34,031 - __main__ - INFO - 가격 데이터 가져오기 성공: 009070.KS, 732개 데이터
2025-07-16 21:48:34,110 - __main__ - INFO - 가격 데이터 가져오기 성공: 009440.KS, 732개 데이터
2025-07-16 21:48:34,272 - __main__ - INFO - 가격 데이터 가져오기 성공: 119650.KS, 732개 데이터
종목 데이터 처리:   3%|▋                      | 79/2757 [00:42<1

2025-07-16 21:48:54,095 - __main__ - WARNING - Deadlock 발생, 0.77s 후 재시도합니다 (1/3)
2025-07-16 21:48:54,446 - __main__ - WARNING - Deadlock 발생, 0.79s 후 재시도합니다 (2/3)
2025-07-16 21:48:54,501 - __main__ - WARNING - Deadlock 발생, 0.88s 후 재시도합니다 (1/3)
종목 데이터 처리:   4%|▉                     | 118/2757 [01:03<28:12,  1.56it/s]2025-07-16 21:48:55,596 - __main__ - WARNING - Deadlock 발생, 0.80s 후 재시도합니다 (2/3)
2025-07-16 21:48:55,620 - __main__ - INFO - 가격 데이터 가져오기 성공: 023150.KS, 732개 데이터
2025-07-16 21:48:55,786 - __main__ - WARNING - Deadlock 발생, 0.69s 후 재시도합니다 (1/3)
종목 데이터 처리:   4%|▉                     | 119/2757 [01:03<29:59,  1.47it/s]2025-07-16 21:48:56,351 - __main__ - INFO - 가격 데이터 가져오기 성공: 035420.KS, 732개 데이터
2025-07-16 21:48:56,441 - __main__ - INFO - 가격 데이터 가져오기 성공: 181710.KS, 732개 데이터
종목 데이터 처리:   4%|▉                     | 123/2757 [01:04<17:10,  2.56it/s]2025-07-16 21:48:57,182 - __main__ - INFO - 가격 데이터 가져오기 성공: 005940.KS, 732개 데이터
2025-07-16 21:48:57,459 - __main__ - INFO - 가격 데이터 가져오기 

종목 데이터 처리:   6%|█▎                    | 162/2757 [01:25<23:03,  1.88it/s]2025-07-16 21:49:17,741 - __main__ - INFO - 가격 데이터 가져오기 성공: 096770.KS, 732개 데이터
2025-07-16 21:49:17,832 - __main__ - INFO - 가격 데이터 가져오기 성공: 096775.KS, 732개 데이터
2025-07-16 21:49:18,295 - __main__ - WARNING - Deadlock 발생, 0.89s 후 재시도합니다 (1/3)
종목 데이터 처리:   6%|█▎                    | 164/2757 [01:26<25:07,  1.72it/s]2025-07-16 21:49:18,903 - __main__ - INFO - 가격 데이터 가져오기 성공: 475150.KS, 313개 데이터
2025-07-16 21:49:19,008 - __main__ - WARNING - Deadlock 발생, 0.98s 후 재시도합니다 (1/3)
2025-07-16 21:49:19,522 - __main__ - WARNING - Deadlock 발생, 0.96s 후 재시도합니다 (1/3)
종목 데이터 처리:   6%|█▎                    | 165/2757 [01:27<28:00,  1.54it/s]2025-07-16 21:49:19,882 - __main__ - WARNING - Deadlock 발생, 0.95s 후 재시도합니다 (1/3)
2025-07-16 21:49:19,905 - __main__ - INFO - 가격 데이터 가져오기 성공: 001510.KS, 732개 데이터
종목 데이터 처리:   6%|█▎                    | 167/2757 [01:28<21:36,  2.00it/s]2025-07-16 21:49:20,497 - __main__ - INFO - 가격 데이터 가져오기 성공: 0015

...

2025-07-16 22:08:53,773 - __main__ - INFO - 가격 데이터 가져오기 성공: 005670.KQ, 722개 데이터
종목 데이터 처리:  93%|███████████████████▌ | 2568/2757 [21:03<01:47,  1.75it/s]2025-07-16 22:08:55,792 - __main__ - WARNING - Deadlock 발생, 0.86s 후 재시도합니다 (1/3)
2025-07-16 22:08:55,876 - __main__ - WARNING - Deadlock 발생, 0.66s 후 재시도합니다 (1/3)
2025-07-16 22:08:56,076 - __main__ - INFO - 가격 데이터 가져오기 성공: 007330.KQ, 729개 데이터
종목 데이터 처리:  93%|███████████████████▌ | 2569/2757 [21:04<01:48,  1.74it/s]2025-07-16 22:08:56,489 - __main__ - INFO - 가격 데이터 가져오기 성공: 093380.KQ, 721개 데이터
2025-07-16 22:08:57,044 - __main__ - WARNING - Deadlock 발생, 0.78s 후 재시도합니다 (1/3)
종목 데이터 처리:  93%|███████████████████▌ | 2570/2757 [21:04<02:07,  1.47it/s]2025-07-16 22:08:57,217 - __main__ - WARNING - Deadlock 발생, 0.94s 후 재시도합니다 (2/3)
2025-07-16 22:08:57,559 - __main__ - INFO - 가격 데이터 가져오기 성공: 023900.KQ, 728개 데이터
종목 데이터 처리:  93%|███████████████████▌ | 2572/2757 [21:05<01:38,  1.87it/s]2025-07-16 22:08:58,050 - __main__ - WARNING - Deadlock 발생, 0.80

종목 데이터 처리:  95%|███████████████████▊ | 2608/2757 [21:22<01:10,  2.10it/s]2025-07-16 22:09:15,500 - __main__ - INFO - 가격 데이터 가져오기 성공: 128660.KQ, 731개 데이터
2025-07-16 22:09:15,885 - __main__ - WARNING - Deadlock 발생, 0.63s 후 재시도합니다 (1/3)
2025-07-16 22:09:16,209 - __main__ - WARNING - Deadlock 발생, 0.70s 후 재시도합니다 (1/3)
종목 데이터 처리:  95%|███████████████████▊ | 2609/2757 [21:24<01:37,  1.52it/s]2025-07-16 22:09:16,509 - __main__ - WARNING - Deadlock 발생, 0.63s 후 재시도합니다 (1/3)
2025-07-16 22:09:16,715 - __main__ - INFO - 가격 데이터 가져오기 성공: 006140.KQ, 728개 데이터
2025-07-16 22:09:17,155 - __main__ - WARNING - Deadlock 발생, 0.74s 후 재시도합니다 (2/3)
종목 데이터 처리:  95%|███████████████████▉ | 2610/2757 [21:24<01:45,  1.39it/s]2025-07-16 22:09:17,305 - __main__ - WARNING - Deadlock 발생, 0.89s 후 재시도합니다 (1/3)
2025-07-16 22:09:17,512 - __main__ - WARNING - Deadlock 발생, 0.79s 후 재시도합니다 (1/3)
2025-07-16 22:09:17,535 - __main__ - INFO - 가격 데이터 가져오기 성공: 376180.KQ, 731개 데이터
종목 데이터 처리:  95%|███████████████████▉ | 2611/2757 [21:25

종목 데이터 처리:  96%|████████████████████▏| 2650/2757 [21:42<00:40,  2.66it/s]2025-07-16 22:09:34,304 - __main__ - INFO - 가격 데이터 가져오기 성공: 256840.KQ, 731개 데이터
2025-07-16 22:09:34,478 - __main__ - INFO - 가격 데이터 가져오기 성공: 219750.KQ, 732개 데이터
종목 데이터 처리:  96%|████████████████████▏| 2651/2757 [21:42<00:44,  2.39it/s]2025-07-16 22:09:35,162 - __main__ - INFO - 가격 데이터 가져오기 성공: 025550.KQ, 731개 데이터
2025-07-16 22:09:35,459 - __main__ - WARNING - Deadlock 발생, 0.70s 후 재시도합니다 (1/3)
종목 데이터 처리:  96%|████████████████████▏| 2652/2757 [21:43<00:53,  1.97it/s]2025-07-16 22:09:35,862 - __main__ - INFO - 가격 데이터 가져오기 성공: 017890.KQ, 730개 데이터
2025-07-16 22:09:36,377 - __main__ - WARNING - Deadlock 발생, 0.88s 후 재시도합니다 (1/3)
2025-07-16 22:09:36,517 - __main__ - WARNING - Deadlock 발생, 0.98s 후 재시도합니다 (1/3)
종목 데이터 처리:  96%|████████████████████▏| 2654/2757 [21:44<01:07,  1.52it/s]2025-07-16 22:09:37,421 - __main__ - WARNING - Deadlock 발생, 0.86s 후 재시도합니다 (1/3)
2025-07-16 22:09:37,553 - __main__ - INFO - 가격 데이터 가져오기 성공: 0414

2025-07-16 22:09:54,328 - __main__ - WARNING - Deadlock 발생, 0.71s 후 재시도합니다 (1/3)
종목 데이터 처리:  98%|████████████████████▍| 2691/2757 [22:02<00:37,  1.75it/s]2025-07-16 22:09:54,859 - __main__ - INFO - 가격 데이터 가져오기 성공: 107640.KQ, 732개 데이터
2025-07-16 22:09:55,474 - __main__ - WARNING - Deadlock 발생, 0.70s 후 재시도합니다 (1/3)
종목 데이터 처리:  98%|████████████████████▌| 2692/2757 [22:03<00:46,  1.41it/s]2025-07-16 22:09:55,665 - __main__ - WARNING - Deadlock 발생, 0.66s 후 재시도합니다 (2/3)
2025-07-16 22:09:56,018 - __main__ - INFO - 가격 데이터 가져오기 성공: 079170.KQ, 730개 데이터
2025-07-16 22:09:56,160 - __main__ - WARNING - Deadlock 발생, 0.74s 후 재시도합니다 (1/3)
종목 데이터 처리:  98%|████████████████████▌| 2693/2757 [22:04<00:44,  1.45it/s]2025-07-16 22:09:56,590 - __main__ - INFO - 가격 데이터 가져오기 성공: 054920.KQ, 731개 데이터
2025-07-16 22:09:56,605 - __main__ - INFO - 가격 데이터 가져오기 성공: 457370.KQ, 178개 데이터
종목 데이터 처리:  98%|████████████████████▌| 2696/2757 [22:04<00:26,  2.26it/s]2025-07-16 22:09:57,186 - __main__ - INFO - 가격 데이터 가져오기 성공: 0026

종목 데이터 처리:  99%|████████████████████▊| 2734/2757 [22:23<00:13,  1.74it/s]2025-07-16 22:10:15,997 - __main__ - INFO - 가격 데이터 가져오기 성공: 115160.KQ, 730개 데이터
2025-07-16 22:10:16,062 - __main__ - WARNING - Deadlock 발생, 0.55s 후 재시도합니다 (1/3)
2025-07-16 22:10:16,225 - __main__ - WARNING - Deadlock 발생, 0.76s 후 재시도합니다 (2/3)
2025-07-16 22:10:16,298 - __main__ - WARNING - Deadlock 발생, 0.89s 후 재시도합니다 (1/3)
종목 데이터 처리:  99%|████████████████████▊| 2737/2757 [22:24<00:08,  2.38it/s]2025-07-16 22:10:16,912 - __main__ - INFO - 가격 데이터 가져오기 성공: 175140.KQ, 731개 데이터
2025-07-16 22:10:17,207 - __main__ - WARNING - Deadlock 발생, 0.68s 후 재시도합니다 (1/3)
2025-07-16 22:10:17,266 - __main__ - INFO - 가격 데이터 가져오기 성공: 200670.KQ, 731개 데이터
2025-07-16 22:10:17,425 - __main__ - WARNING - Deadlock 발생, 0.94s 후 재시도합니다 (2/3)
종목 데이터 처리:  99%|████████████████████▊| 2739/2757 [22:25<00:07,  2.46it/s]2025-07-16 22:10:18,044 - __main__ - INFO - 가격 데이터 가져오기 성공: 263920.KQ, 673개 데이터
2025-07-16 22:10:18,071 - __main__ - INFO - 가격 데이터 가져오기 

# 옵션

In [70]:
# 또는 업데이트
run_database_task("update", days=30)

In [86]:
# 데이터베이스 연결 및 내용 확인
from sqlalchemy import create_engine, inspect
import pandas as pd

# 데이터베이스 URI
# 올바른 PostgreSQL 연결 문자열 사용
DB_URI = 'mysql+pymysql://root@localhost:3306/finance_db'

# DB_URI = 'sqlite:///finance_data.db'  # SQLite 설정한 경우

# 엔진 생성
engine = create_engine(DB_URI, echo=False)

# 인스펙터 생성
inspector = inspect(engine)

# 테이블 목록 확인
tables = inspector.get_table_names()
print("데이터베이스 테이블 목록:")
for table in tables:
    print(f"- {table}")

# 각 테이블의 데이터 수 확인
for table in tables:
    query = f"SELECT COUNT(*) FROM {table}"
    count = pd.read_sql(query, engine).iloc[0, 0]
    print(f"{table}: {count}개 레코드")

# 샘플 데이터 확인 (테이블별)
# 종목 정보
if 'stocks' in tables:
    stocks_df = pd.read_sql("SELECT * FROM stocks LIMIT 10", engine)
    print("\n종목 정보 샘플:")
    print(stocks_df)

# 가격 데이터
if 'daily_prices' in tables:
    prices_df = pd.read_sql("SELECT * FROM daily_prices LIMIT 10", engine)
    print("\n가격 데이터 샘플:")
    print(prices_df)

# 기술적 지표
if 'technical_indicators' in tables:
    indicators_df = pd.read_sql("SELECT * FROM technical_indicators LIMIT 10", engine)
    print("\n기술적 지표 샘플:")
    print(indicators_df)

# 시장 지수
if 'market_indices' in tables:
    indices_df = pd.read_sql("SELECT * FROM market_indices LIMIT 10", engine)
    print("\n시장 지수 샘플:")
    print(indices_df)

# 시장 통계
if 'market_stats' in tables:
    stats_df = pd.read_sql("SELECT * FROM market_stats LIMIT 10", engine)
    print("\n시장 통계 샘플:")
    print(stats_df)

데이터베이스 테이블 목록:
- daily_prices
- market_indices
- market_stats
- stocks
- technical_indicators
daily_prices: 1869013개 레코드
market_indices: 1463개 레코드
market_stats: 2196개 레코드
stocks: 2757개 레코드
technical_indicators: 1815169개 레코드

종목 정보 샘플:
   stock_id     symbol krx_code      name market                  sector  \
0         1  095570.KS   095570    AJ네트웍스  KOSPI             Industrials   
1         2  006840.KS   006840     AK홀딩스  KOSPI         Basic Materials   
2         3  027410.KS   027410       BGF  KOSPI         Basic Materials   
3         4  282330.KS   282330    BGF리테일  KOSPI      Consumer Defensive   
4         5  138930.KS   138930   BNK금융지주  KOSPI      Financial Services   
5         6  001460.KS   001460       BYC  KOSPI       Consumer Cyclical   
6         7  001465.KS   001465      BYC우  KOSPI       Consumer Cyclical   
7         8  001040.KS   001040        CJ  KOSPI             Industrials   
8         9  079160.KS   079160    CJ CGV  KOSPI  Communication Services   
9    

# 데이터베이스 삭제 후 재실행

In [45]:
# PostgreSQL 데이터베이스 연결
import pymysql

# 연결 설정
conn = pymysql.connect(
    host='localhost',
    user='root',
    password='', 
    database='finance_db',
    charset='utf8mb4'
)
conn.autocommit = True
cursor = conn.cursor()

# 기존 테이블 삭제 (CASCADE 옵션으로 관련 제약조건도 함께 삭제)
cursor.execute("DROP TABLE IF EXISTS daily_prices CASCADE")
cursor.execute("DROP TABLE IF EXISTS technical_indicators CASCADE")
cursor.execute("DROP TABLE IF EXISTS market_indices CASCADE")
cursor.execute("DROP TABLE IF EXISTS market_stats CASCADE")
cursor.execute("DROP TABLE IF EXISTS stocks CASCADE")

conn.close()
print("기존 테이블이 삭제되었습니다.")

2025-07-16 21:20:55,092 - __main__ - INFO - 가격 데이터 가져오기 성공: 007460.KS, 732개 데이터
2025-07-16 21:20:56,149 - __main__ - ERROR - 가격 데이터 저장 오류 (종목 ID: 600): (pymysql.err.ProgrammingError) (1146, "Table 'finance_db.daily_prices' doesn't exist")
[SQL: SELECT daily_prices.price_id AS daily_prices_price_id, daily_prices.stock_id AS daily_prices_stock_id, daily_prices.date AS daily_prices_date, daily_prices.open_price AS daily_prices_open_price, daily_prices.high_price AS daily_prices_high_price, daily_prices.low_price AS daily_prices_low_price, daily_prices.close_price AS daily_prices_close_price, daily_prices.adjusted_close AS daily_prices_adjusted_close, daily_prices.volume AS daily_prices_volume, daily_prices.`change` AS daily_prices_change, daily_prices.change_rate AS daily_prices_change_rate, daily_prices.created_at AS daily_prices_created_at, daily_prices.updated_at AS daily_prices_updated_at 
FROM daily_prices 
WHERE daily_prices.stock_id = %(stock_id_1)s AND daily_prices.date = %(date_1

2025-07-16 21:20:56,153 - __main__ - ERROR - 기술적 지표 계산 오류 (종목 ID: 591): (pymysql.err.ProgrammingError) (1146, "Table 'finance_db.daily_prices' doesn't exist")
[SQL: SELECT daily_prices.price_id AS daily_prices_price_id, daily_prices.stock_id AS daily_prices_stock_id, daily_prices.date AS daily_prices_date, daily_prices.open_price AS daily_prices_open_price, daily_prices.high_price AS daily_prices_high_price, daily_prices.low_price AS daily_prices_low_price, daily_prices.close_price AS daily_prices_close_price, daily_prices.adjusted_close AS daily_prices_adjusted_close, daily_prices.volume AS daily_prices_volume, daily_prices.`change` AS daily_prices_change, daily_prices.change_rate AS daily_prices_change_rate, daily_prices.created_at AS daily_prices_created_at, daily_prices.updated_at AS daily_prices_updated_at 
FROM daily_prices 
WHERE daily_prices.stock_id = %(stock_id_1)s ORDER BY daily_prices.date]
[parameters: {'stock_id_1': 591}]
(Background on this error at: https://sqlalche.me/

기존 테이블이 삭제되었습니다.


2025-07-16 21:20:56,355 - __main__ - INFO - 가격 데이터 가져오기 성공: 278470.KS, 335개 데이터
2025-07-16 21:20:56,364 - __main__ - ERROR - 가격 데이터 저장 오류 (종목 ID: 603): (pymysql.err.ProgrammingError) (1146, "Table 'finance_db.daily_prices' doesn't exist")
[SQL: SELECT daily_prices.price_id AS daily_prices_price_id, daily_prices.stock_id AS daily_prices_stock_id, daily_prices.date AS daily_prices_date, daily_prices.open_price AS daily_prices_open_price, daily_prices.high_price AS daily_prices_high_price, daily_prices.low_price AS daily_prices_low_price, daily_prices.close_price AS daily_prices_close_price, daily_prices.adjusted_close AS daily_prices_adjusted_close, daily_prices.volume AS daily_prices_volume, daily_prices.`change` AS daily_prices_change, daily_prices.change_rate AS daily_prices_change_rate, daily_prices.created_at AS daily_prices_created_at, daily_prices.updated_at AS daily_prices_updated_at 
FROM daily_prices 
WHERE daily_prices.stock_id = %(stock_id_1)s AND daily_prices.date = %(date_1

2025-07-16 21:20:56,550 - __main__ - ERROR - 가격 데이터 저장 오류 (종목 ID: 607): (pymysql.err.ProgrammingError) (1146, "Table 'finance_db.daily_prices' doesn't exist")
[SQL: SELECT daily_prices.price_id AS daily_prices_price_id, daily_prices.stock_id AS daily_prices_stock_id, daily_prices.date AS daily_prices_date, daily_prices.open_price AS daily_prices_open_price, daily_prices.high_price AS daily_prices_high_price, daily_prices.low_price AS daily_prices_low_price, daily_prices.close_price AS daily_prices_close_price, daily_prices.adjusted_close AS daily_prices_adjusted_close, daily_prices.volume AS daily_prices_volume, daily_prices.`change` AS daily_prices_change, daily_prices.change_rate AS daily_prices_change_rate, daily_prices.created_at AS daily_prices_created_at, daily_prices.updated_at AS daily_prices_updated_at 
FROM daily_prices 
WHERE daily_prices.stock_id = %(stock_id_1)s AND daily_prices.date = %(date_1)s 
 LIMIT %(param_1)s]
[parameters: {'stock_id_1': 607, 'date_1': datetime.date

...

2025-07-16 21:21:56,282 - __main__ - ERROR - 가격 데이터 저장 오류 (종목 ID: 2725): (pymysql.err.ProgrammingError) (1146, "Table 'finance_db.daily_prices' doesn't exist")
[SQL: SELECT daily_prices.price_id AS daily_prices_price_id, daily_prices.stock_id AS daily_prices_stock_id, daily_prices.date AS daily_prices_date, daily_prices.open_price AS daily_prices_open_price, daily_prices.high_price AS daily_prices_high_price, daily_prices.low_price AS daily_prices_low_price, daily_prices.close_price AS daily_prices_close_price, daily_prices.adjusted_close AS daily_prices_adjusted_close, daily_prices.volume AS daily_prices_volume, daily_prices.`change` AS daily_prices_change, daily_prices.change_rate AS daily_prices_change_rate, daily_prices.created_at AS daily_prices_created_at, daily_prices.updated_at AS daily_prices_updated_at 
FROM daily_prices 
WHERE daily_prices.stock_id = %(stock_id_1)s AND daily_prices.date = %(date_1)s 
 LIMIT %(param_1)s]
[parameters: {'stock_id_1': 2725, 'date_1': datetime.da

2025-07-16 21:21:56,446 - __main__ - ERROR - 가격 데이터 저장 실패: 화인써키트 (127980.KQ)
2025-07-16 21:21:56,483 - __main__ - INFO - 가격 데이터 가져오기 성공: 290270.KQ, 729개 데이터
2025-07-16 21:21:56,489 - __main__ - ERROR - 가격 데이터 저장 오류 (종목 ID: 2739): (pymysql.err.ProgrammingError) (1146, "Table 'finance_db.daily_prices' doesn't exist")
[SQL: SELECT daily_prices.price_id AS daily_prices_price_id, daily_prices.stock_id AS daily_prices_stock_id, daily_prices.date AS daily_prices_date, daily_prices.open_price AS daily_prices_open_price, daily_prices.high_price AS daily_prices_high_price, daily_prices.low_price AS daily_prices_low_price, daily_prices.close_price AS daily_prices_close_price, daily_prices.adjusted_close AS daily_prices_adjusted_close, daily_prices.volume AS daily_prices_volume, daily_prices.`change` AS daily_prices_change, daily_prices.change_rate AS daily_prices_change_rate, daily_prices.created_at AS daily_prices_created_at, daily_prices.updated_at AS daily_prices_updated_at 
FROM daily_prices 

2025-07-16 21:21:56,639 - __main__ - ERROR - 가격 데이터 저장 실패: 휴맥스 (115160.KQ)
2025-07-16 21:21:56,639 - __main__ - ERROR - 가격 데이터 저장 오류 (종목 ID: 2738): (pymysql.err.ProgrammingError) (1146, "Table 'finance_db.daily_prices' doesn't exist")
[SQL: SELECT daily_prices.price_id AS daily_prices_price_id, daily_prices.stock_id AS daily_prices_stock_id, daily_prices.date AS daily_prices_date, daily_prices.open_price AS daily_prices_open_price, daily_prices.high_price AS daily_prices_high_price, daily_prices.low_price AS daily_prices_low_price, daily_prices.close_price AS daily_prices_close_price, daily_prices.adjusted_close AS daily_prices_adjusted_close, daily_prices.volume AS daily_prices_volume, daily_prices.`change` AS daily_prices_change, daily_prices.change_rate AS daily_prices_change_rate, daily_prices.created_at AS daily_prices_created_at, daily_prices.updated_at AS daily_prices_updated_at 
FROM daily_prices 
WHERE daily_prices.stock_id = %(stock_id_1)s AND daily_prices.date = %(date_1)s 


2025-07-16 21:21:56,823 - __main__ - ERROR - 가격 데이터 저장 실패: 휴온스 (243070.KQ)
2025-07-16 21:21:56,842 - __main__ - INFO - 가격 데이터 가져오기 성공: 145020.KQ, 731개 데이터
2025-07-16 21:21:56,844 - __main__ - ERROR - 가격 데이터 저장 오류 (종목 ID: 2752): (pymysql.err.ProgrammingError) (1146, "Table 'finance_db.daily_prices' doesn't exist")
[SQL: SELECT daily_prices.price_id AS daily_prices_price_id, daily_prices.stock_id AS daily_prices_stock_id, daily_prices.date AS daily_prices_date, daily_prices.open_price AS daily_prices_open_price, daily_prices.high_price AS daily_prices_high_price, daily_prices.low_price AS daily_prices_low_price, daily_prices.close_price AS daily_prices_close_price, daily_prices.adjusted_close AS daily_prices_adjusted_close, daily_prices.volume AS daily_prices_volume, daily_prices.`change` AS daily_prices_change, daily_prices.change_rate AS daily_prices_change_rate, daily_prices.created_at AS daily_prices_created_at, daily_prices.updated_at AS daily_prices_updated_at 
FROM daily_prices 
W

2025-07-16 21:21:57,012 - __main__ - ERROR - 가격 데이터 저장 실패: 흥구석유 (024060.KQ)
2025-07-16 21:21:57,110 - yfinance - ERROR - $331740.KQ: possibly delisted; no price data found  (1d 2022-07-17 -> 2025-07-16)
2025-07-16 21:21:57,111 - __main__ - WARNING - 가격 데이터가 없습니다: 331740.KQ
2025-07-16 21:21:57,111 - __main__ - WARNING - 가격 데이터 없음: 아우토크립트 (331740.KQ)
